In [1]:
# Import necessary libraries
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tkinter as tk
from tkinter import filedialog

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [2]:
# Set the base directory
base_dir = os.getcwd()  # Get the current working directory

# Construct the path to the dataset
dataset_dir = os.path.join(base_dir, 'dataset')

# Construct the path to the training directory
train_dir = os.path.join(dataset_dir, 'train')

# Construct the paths to each subdirectory for training
train_headtop_dir = os.path.join(train_dir, 'headtop')
train_helmet_dir = os.path.join(train_dir, 'helmet')
train_hoodie_dir = os.path.join(train_dir, 'hoodie')
train_no_headware_dir = os.path.join(train_dir, 'no_headwear')

# Print the total number of images in each training subdirectory
if os.path.exists(train_headtop_dir):
    print('Total training headtop images:', len(os.listdir(train_headtop_dir)))
else:
    print(f"Directory {train_headtop_dir} does not exist")

if os.path.exists(train_helmet_dir):
    print('Total training helmet images:', len(os.listdir(train_helmet_dir)))
else:
    print(f"Directory {train_helmet_dir} does not exist")

if os.path.exists(train_hoodie_dir):
    print('Total training hoodie images:', len(os.listdir(train_hoodie_dir)))
else:
    print(f"Directory {train_hoodie_dir} does not exist")

if os.path.exists(train_no_headware_dir):
    print('Total training no headware images:', len(os.listdir(train_no_headware_dir)))
else:
    print(f"Directory {train_no_headware_dir} does not exist")



Total training headtop images: 993
Total training helmet images: 973
Total training hoodie images: 991
Total training no headware images: 962


In [4]:
# Construct the path to the validation directory
val_dir = os.path.join(dataset_dir, 'val')

# Construct the paths to each subdirectory for validation
val_headtop_dir = os.path.join(val_dir, 'headtop')
val_helmet_dir = os.path.join(val_dir, 'helmet')
val_hoodie_dir = os.path.join(val_dir, 'hoodie')
val_no_headware_dir = os.path.join(val_dir, 'no_headwear')

# Print the total number of images in each validation subdirectory
if os.path.exists(val_headtop_dir):
    print('Total validation headtop images:', len(os.listdir(val_headtop_dir)))
else:
    print(f"Directory {val_headtop_dir} does not exist")

if os.path.exists(val_helmet_dir):
    print('Total validation helmet images:', len(os.listdir(val_helmet_dir)))
else:
    print(f"Directory {val_helmet_dir} does not exist")

if os.path.exists(val_hoodie_dir):
    print('Total validation hoodie images:', len(os.listdir(val_hoodie_dir)))
else:
    print(f"Directory {val_hoodie_dir} does not exist")

if os.path.exists(val_no_headware_dir):
    print('Total validation no headware images:', len(os.listdir(val_no_headware_dir)))
else:
    print(f"Directory {val_no_headware_dir} does not exist")

Total validation headtop images: 100
Total validation helmet images: 98
Total validation hoodie images: 100
Total validation no headware images: 97


In [5]:
# Create data generators
val_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescale the images

validation_generator = val_datagen.flow_from_directory(
    val_dir,  # Use the validation directory
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'  # Change to categorical for multi-class classification
)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 395 images belonging to 4 classes.
Found 3919 images belonging to 4 classes.


In [12]:
# Define the simplified model with MaxPooling2D and Dropout
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(4, activation='softmax')
])
model.summary()

c:\Users\JanTilles\anaconda3\envs\ml-env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 298, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 341056)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    43,655,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,675,204 (166.61 MB)

 Trainable params: 43,675,204 (166.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile multi-class classification model
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [14]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - acc: 0.3191 - loss: 9.2589

c:\Users\JanTilles\anaconda3\envs\ml-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/123 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - acc: 0.3194 - loss: 9.2109 - val_acc: 0.4380 - val_loss: 1.2334
Epoch 2/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - acc: 0.4109 - loss: 1.2930 - val_acc: 0.4835 - val_loss: 1.1634
Epoch 3/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - acc: 0.4781 - loss: 1.2125 - val_acc: 0.5392 - val_loss: 1.1111
Epoch 4/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 314s 3s/step - acc: 0.5109 - loss: 1.1621 - val_acc: 0.5722 - val_loss: 0.9898
Epoch 5/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 379s 3s/step - acc: 0.5209 - loss: 1.1312 - val_acc: 0.6000 - val_loss: 0.9808
Epoch 6/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - acc: 0.5513 - loss: 1.0650 - val_acc: 0.6506 - val_loss: 0.8974
Epoch 7/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - acc: 0.5702 - loss: 1.0431 - val_acc: 0.6203 - val_loss: 0.8875
Epoch 8/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - acc: 0.5584 - loss: 1.0268 - val_acc: 0.6000 - val_loss: 0.9589
Epoch 9/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 109s 884ms/step - a

In [15]:
# Save the model in the native Keras format
model_path = os.path.join(base_dir, 'model.keras')  # Construct the file path
model.save(model_path)  # Save the model

In [5]:
# Set the base directory
script_dir = os.getcwd()  # Get the current working directory

# Load your pre-trained model
# Replace 'path/to/your/model.keras' with the actual path to your model file
model_path = os.path.join(script_dir, 'model.keras')
model = load_model(model_path)

# Create a Tkinter root window (it will be hidden)
root = tk.Tk()
root.withdraw()

# Open a file dialog and select files
file_paths = filedialog.askopenfilenames()

# Define the target size for the images (should match the input size of the model)
target_size = (512, 512)

for path in file_paths:
    # Predicting images
    img = image.load_img(path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    
    # Assuming the model outputs probabilities for each class
    class_index = np.argmax(classes[0])
    class_labels = ['Headtop', 'Helmet', 'Hoodie', 'No headwear']  
    print(f"{path} is a {class_labels[class_index]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
[4.104521e-33 1.000000e+00 7.993007e-16 0.000000e+00]
C:/Users/JanTilles/Documents/Turku AMK/Components and apps of AI/Code/TUAS-ComponentsOfAI/assignment/dataset/test/no_headwear/10.jpg is a Helmet


In [16]:
# Set the base directory
script_dir = os.getcwd()  # Get the current working directory

# Load your pre-trained model
# Replace 'path/to/your/model.keras' with the actual path to your model file
model_path = os.path.join(script_dir, 'model.keras')
model = load_model(model_path)

# Define class labels
class_labels = ['Headtop', 'Helmet', 'Hoodie', 'No headwear']

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Define the target size for the images
target_size = (300, 300)

print("Press 'q' to exit the video feed.")

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame. Exiting...")
            break

        # Preprocess the frame
        resized_frame = cv2.resize(frame, target_size)  # Resize to match model input
        img_array = np.expand_dims(resized_frame, axis=0) / 255.0  # Normalize and add batch dimension

        # Make prediction
        predictions = model.predict(img_array)
        class_index = np.argmax(predictions[0])
        prediction_label = class_labels[class_index]
        confidence = predictions[0][class_index] * 100

        # Display the prediction on the frame
        cv2.putText(frame, f"{prediction_label} ({confidence:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('Hat Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProgram interrupted by the user. Exiting...")

finally:
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released, video window closed.")


Press 'q' to exit the video feed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━